### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.G_spec = self.__gini_spec
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.G_spec = self.__entropy_spec
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.G_spec = self.__misclass_spec
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - ((l_c ** 2) / (l_s * (l_s + r_s)) + (r_c ** 2) / (r_s * (l_s + r_s))).sum(axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return - (l_c / l_s * np.log2(l_c / l_s)).sum(axis=1) - (r_c / r_s * np.log2(r_c / r_s)).sum(axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c / (l_s + r_s)).max(axis=1) - (r_c / (l_s + r_s)).max(axis=1)
    
    def __gini_spec(self, y):
        return 1 - (np.unique(y, return_counts=True)[1] ** 2 / y.size ** 2).sum()
    
    def __entropy_spec(self, y):
        return -(((np.unique(y, return_counts=True)[1] / y.shape[0])
                  * np.log2(np.unique(y, return_counts=True)[1] / y.shape[0])).sum())
        
    def __misclass_spec(self, y):
        return 1 - (np.unique(y, return_counts=True)[1] / y.size).sum().max()
        
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:max(1, int(np.sqrt(n_feature)))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:max(1, int(np.log2(n_feature)))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(max(1, n_feature))
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_sort, y_sort=self.__sort_samples(x,y)
        # индексы с изменением класса
        split_indexes = np.where(y_sort[:-1] != y_sort[1:])[0] + 1
        # если класс не меняется то просто возвращаемся
        if len(split_indexes) == 0:
            return np.inf, None
        split_classes = np.zeros((split_indexes.shape[0], self.num_class))
        split_classes[np.arange(split_indexes.shape[0]), y_sort[split_indexes - 1]] = 1
        elements = split_indexes - np.append([0], split_indexes[:-1])
        split_classes = split_classes * elements.reshape(-1, 1)
        # количество классов и объектов слева и справа соответственно
        left_class_count = np.cumsum(split_classes, axis=0)
        right_class_count = np.bincount(y_sort, minlength=self.num_class) - left_class_count
        left_size = split_indexes.reshape(split_indexes.shape[0], 1)
        right_size = y_sort.shape[0] - left_size
        # считаем критерий
        clt = self.G_function(left_class_count, left_size, right_class_count, right_size)
        min_index = np.argmin(clt)
        # возвращаем критерий и пороговое значение для него
        thr_i = split_indexes[min_index]
        return clt[min_index], (x_sort[thr_i] + x_sort[thr_i - 1]) * 0.5

    def __fit_node(self, x, y, node_id, depth):
        # создаем листовую вершину, если выполнено условие остановы
        if ((self.max_depth is not None and depth >= self.max_depth) or
                np.bincount(y).argmax() / y.shape[0] >= self.sufficient_share or
                self.min_samples_split > y.shape[0]):
            prob = np.bincount(y).astype(float) / y.shape[0]
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), prob)
            return
        # найдём лучшее разбиение и его параметры
        thr = np.array([self.__find_threshold(x[:, i], y) for i in self.get_feature_ids(x.shape[1])])
        min_id = thr[:, 0].argmin()
        best_thr = thr[min_id, 1]
        # если можем создать узел, идём дальше, иначе делаем листовую вершину
        if best_thr is None:
            prob = np.bincount(y).astype(float) / y.shape[0]
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), prob)
        else:
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, min_id, best_thr)
            # если не можем разделить, создаём лист
            if x_left.shape[0] == 0 or x_right.shape[0] == 0:
                prob = np.bincount(y).astype(float) / y.shape[0]
                self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), prob)
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE, min_id, best_thr)
                self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
                self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)
            # вычисляем feature_importance
            pred = y.shape[0] * self.G_spec(y)
            left = y_left.shape[0] * self.G_spec(y_left)
            right = y_right.shape[0] * self.G_spec(y_right)
            self.feature_importances_[min_id] += pred - left - right
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= y.shape[0]

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 1.52 ms, sys: 1.04 ms, total: 2.55 ms
Wall time: 1.49 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 9.86 ms, sys: 2.18 ms, total: 12 ms
Wall time: 10.2 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8851851851851852

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8851851851851852

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', encoding='latin1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(8)
memory usage: 12.5+ MB


In [10]:
# Удалим нечисленные столбцы
for i in df.columns:
    if (type(df[i][0]) == np.float64 or
            type(df[i][0]) == np.int64 or
            type(df[i][0]) == float):
        continue
    df.drop(i, axis=1, inplace=True)

In [11]:
# Удалим столбец, если слишком много None
for i in df.columns:
    if df[i].isnull().sum() > int(0.1 * len(df)):
        df.drop(i, axis=1, inplace=True)

In [12]:
# Заменим None на невозмодные значения
df = df.fillna(-90000)
# Разделим на train и test
y = df.match.values
X = df.drop(['dec','dec_o','attr','attr_o','fun','fun_o','int_corr', 'match'], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


## Проверка скорости работы на Speed Dating Data 

In [13]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [14]:
%time clf.fit(X_train, y_train)

CPU times: user 331 ms, sys: 6.62 ms, total: 337 ms
Wall time: 226 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [15]:
%time my_clf.fit(X_train, y_train)

CPU times: user 2.07 s, sys: 40.1 ms, total: 2.11 s
Wall time: 2.21 s


## Проверка качества работы на Speed Dating Data

In [16]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.654480483782298

In [17]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.6192459718191489

## Задание 3

## Задание 4

In [18]:
print(clf.feature_importances_.argsort()[:-10:-1])
print(my_clf.feature_importances_.argsort()[:-10:-1])

[23 73 74 24 19  9 10  7  8]
[23 73  0  1 24 74  8  7  9]


## Задание 5

In [19]:
param = {
    "max_depth": [i for i in range(2, 20)],
    "min_samples_split": [i for i in range(2, 10)],
    "n_estimators": [i for i in range(50, 150,10)],
}

RSearch = RandomizedSearchCV(RandomForestClassifier(random_state=777), param_distributions=param, n_iter=75)
RSearch.fit(X_train, y_train)

print(RSearch.best_params_)


/Users/koteuka/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 100, 'min_samples_split': 2, 'max_depth': 15}
